In [1]:
import mxnet as mx
from mxnet import autograd, gluon, nd
from mxnet.gluon import nn, rnn, Block
from mxnet.contrib import text

import collections
import datetime
import pickle
import numpy as np

In [2]:
epochs = 50
epoch_period = 1

learning_rate = 0.01
batch_size = 64

max_seq_len = 20
max_output_len = 20

encoder_num_layers = 2
decoder_num_layers = 4

encoder_drop_prob = 0.1
decoder_drop_prob = 0.1

encoder_hidden_dim = 256
decoder_hidden_dim = 512
alignment_dim = 512

ctx = mx.gpu(1)

PAD = '<pad>'
BOS = '<bos>'
EOS = '<eos>'


In [3]:
dataset_size = 3200
input_tokens = []
output_tokens = []
input_seqs = []
output_seqs = []

with open('/mnt/sharedisk/tohka/emotion/stc_weibo_train_post') as f:
    inputs = f.readlines()
with open('/mnt/sharedisk/tohka/emotion/stc_weibo_train_response') as f:
    outputs = f.readlines()
inputs = [line[:-1] for line in inputs]
outputs = [line[:-1] for line in outputs]
for i in range(dataset_size):
    input_seq = inputs[i]
    output_seq = outputs[i]
    cur_input_tokens = input_seq.split(' ')
    cur_output_tokens = output_seq.split(' ')

    if len(cur_input_tokens) < max_seq_len and len(cur_output_tokens) < max_seq_len:
        input_tokens.extend(cur_input_tokens)
        cur_input_tokens.append(EOS)
        while len(cur_input_tokens) < max_seq_len:
            cur_input_tokens.append(PAD)
        input_seqs.append(cur_input_tokens)

        output_tokens.extend(cur_output_tokens)
        cur_output_tokens.append(EOS)
        while len(cur_output_tokens) < max_seq_len:
            cur_output_tokens.append(PAD)
        output_seqs.append(cur_output_tokens)

input_vocab = text.vocab.Vocabulary(collections.Counter(input_tokens), 
                                    reserved_tokens=[PAD,BOS,EOS])
output_vocab = text.vocab.Vocabulary(collections.Counter(output_tokens),
                                    reserved_tokens=[PAD,BOS,EOS])

print(len(input_vocab))
print(len(output_vocab))
del inputs
del outputs

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/sharedisk/tohka/emotion/stc_weibo_train_post'

In [4]:
X = nd.zeros((len(input_seqs), max_seq_len), ctx = ctx)
Y = nd.zeros((len(output_seqs), max_seq_len), ctx = ctx)
for i in range(len(input_seqs)):
    X[i] = nd.array(input_vocab.to_indices(input_seqs[i]), ctx = ctx)
    Y[i] = nd.array(output_vocab.to_indices(output_seqs[i]), ctx = ctx)
dataset = gluon.data.ArrayDataset(X, Y)

-------------------------------------------------

In [4]:
class Encoder(Block):
    def __init__(self, input_dim, hidden_dim, num_layers, drop_prob, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        with self.name_scope():
            self.embedding = nn.Embedding(input_dim, hidden_dim)
            self.dropout = nn.Dropout(drop_prob)
            self.rnn = rnn.GRU(hidden_dim, num_layers, dropout=drop_prob, input_size=hidden_dim)
    
    def forward(self, inputs, state):
        emb = self.embedding(inputs).swapaxes(0,1)
        emb = self.dropout(emb)
        output, state = self.rnn(emb, state)
        return output, state

    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

In [5]:
class Decoder(Block):
    """含注意力机制的解码器"""
    def __init__(self, hidden_dim, output_dim, num_layers, max_seq_len,
                 drop_prob, alignment_dim, encoder_hidden_dim, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.max_seq_len = max_seq_len
        self.encoder_hidden_dim = encoder_hidden_dim
        self.hidden_size = hidden_dim
        self.num_layers = num_layers
        with self.name_scope():
            self.embedding = nn.Embedding(output_dim, hidden_dim)
            self.dropout = nn.Dropout(drop_prob)
            # 注意力机制。
            self.attention = nn.Sequential()
            with self.attention.name_scope():
                self.attention.add(nn.Dense(
                    alignment_dim, in_units=hidden_dim + encoder_hidden_dim,
                    activation="tanh", flatten=False))
                self.attention.add(nn.Dense(1, in_units=alignment_dim,
                                            flatten=False))

            self.rnn = rnn.GRU(hidden_dim, num_layers, dropout=drop_prob,
                               input_size=hidden_dim)
            self.out = nn.Dense(output_dim, in_units=hidden_dim, flatten=False)
            self.rnn_concat_input = nn.Dense(
                hidden_dim, in_units=hidden_dim + encoder_hidden_dim,
                flatten=False)

    def forward(self, cur_input, state, encoder_outputs):
        # 当RNN为多层时，取最靠近输出层的单层隐含状态。
        single_layer_state = [state[0][-1].expand_dims(0)]
        encoder_outputs = encoder_outputs.reshape((self.max_seq_len, -1,
                                                   self.encoder_hidden_dim))
        # single_layer_state尺寸: [(1, batch_size, decoder_hidden_dim)]
        # hidden_broadcast尺寸: (max_seq_len, batch_size, decoder_hidden_dim)
        hidden_broadcast = nd.broadcast_axis(single_layer_state[0], axis=0,
                                             size=self.max_seq_len)

        # encoder_outputs_and_hiddens尺寸:
        # (max_seq_len, batch_size, encoder_hidden_dim + decoder_hidden_dim)
        encoder_outputs_and_hiddens = nd.concat(encoder_outputs,
                                                hidden_broadcast, dim=2)

        # energy尺寸: (max_seq_len, batch_size, 1)
        energy = self.attention(encoder_outputs_and_hiddens)

        # batch_attention尺寸: (batch_size, 1, max_seq_len)
        batch_attention = nd.softmax(energy, axis=0).transpose(
            (1, 2, 0))

        # batch_encoder_outputs尺寸: (batch_size, max_seq_len, encoder_hidden_dim)
        batch_encoder_outputs = encoder_outputs.swapaxes(0, 1)

        # decoder_context尺寸: (batch_size, 1, encoder_hidden_dim)
        decoder_context = nd.batch_dot(batch_attention, batch_encoder_outputs)

        # cur_input尺寸: (batch_size,)
        # input_and_context尺寸: (batch_size, 1, encoder_hidden_dim + decoder_hidden_dim)
        input_and_context = nd.concat(nd.expand_dims(self.embedding(cur_input), axis=1),
                                      decoder_context, dim=2)
        # concat_input尺寸: (1, batch_size, decoder_hidden_dim)
        concat_input = self.rnn_concat_input(input_and_context).reshape((1, -1, 0))
        concat_input = self.dropout(concat_input)

        # 当RNN为多层时，用单层隐含状态初始化各个层的隐含状态。
        state = [nd.broadcast_axis(single_layer_state[0], axis=0,
                                   size=self.num_layers)]

        output, state = self.rnn(concat_input, state)
        output = self.dropout(output)
        output = self.out(output).reshape((-3, -1))
        # output尺寸: (batch_size, output_size)
        return output, state

    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

In [6]:
class DecoderInitState(Block):
    """解码器隐含状态的初始化"""
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim, **kwargs):
        super(DecoderInitState, self).__init__(**kwargs)
        with self.name_scope():
            self.dense = nn.Dense(decoder_hidden_dim,
                                  in_units=encoder_hidden_dim,
                                  activation="tanh", flatten=False)

    def forward(self, encoder_state):
        return [self.dense(encoder_state)]

In [7]:
encoder = Encoder(len(input_vocab), encoder_hidden_dim, encoder_num_layers,
                  encoder_drop_prob)
decoder = Decoder(decoder_hidden_dim, len(output_vocab),
                  decoder_num_layers, max_seq_len, decoder_drop_prob,
                  alignment_dim, encoder_hidden_dim)
decoder_init_state = DecoderInitState(encoder_hidden_dim, decoder_hidden_dim)

In [8]:
encoder.collect_params().load('encoder-bless-L118.params', ctx=ctx)
decoder.collect_params().load('decoder-bless-L118.params', ctx=ctx)
decoder_init_state.collect_params().load('decoder-init-state-bless-L118.params', ctx=ctx)

In [9]:
encoder.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
decoder.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
decoder_init_state.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

#encoder.collect_params().load('encoder-epoch50.params', ctx=ctx)
#decoder.collect_params().load('decoder-epoch50.params', ctx=ctx)
#decoder_init_state.collect_params().load('decoder-init-state-epoch50.params', ctx=ctx)



softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [29]:
learning_rate = 0.005 / 8
encoder_optimizer = gluon.Trainer(encoder.collect_params(), 'adam',
                                  {'learning_rate': learning_rate})
decoder_optimizer = gluon.Trainer(decoder.collect_params(), 'adam',
                                  {'learning_rate': learning_rate})
decoder_init_state_optimizer = gluon.Trainer(
    decoder_init_state.collect_params(), 'adam',
    {'learning_rate': learning_rate})

prev_time = datetime.datetime.now()
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

total_loss = 0.0
eos_id = output_vocab.token_to_idx[EOS]
iter_times = 0

for epoch in range(1, epochs + 1):
    for x, y in data_iter:
        real_batch_size = x.shape[0]
        with autograd.record():
            loss = nd.array([0], ctx=ctx)
            valid_length = nd.array([0], ctx=ctx)
            encoder_state = encoder.begin_state(
                func=mx.nd.zeros, batch_size=real_batch_size, ctx=ctx)
            encoder_outputs, encoder_state = encoder(x, encoder_state)

            # encoder_outputs尺寸: (max_seq_len, encoder_hidden_dim)
            encoder_outputs = encoder_outputs.flatten()
            # 解码器的第一个输入为BOS字符。
            decoder_input = nd.array([output_vocab.token_to_idx[BOS]] * real_batch_size,
                                     ctx=ctx)
            mask = nd.ones(shape=(real_batch_size,), ctx=ctx)
            decoder_state = decoder_init_state(encoder_state[0])
            for i in range(max_seq_len):
                decoder_output, decoder_state = decoder(
                    decoder_input, decoder_state, encoder_outputs)
                # 解码器使用当前时刻的预测结果作为下一时刻的输入。
                decoder_input = decoder_output.argmax(axis=1)
                valid_length = valid_length + mask.sum()
                loss = loss + (mask * softmax_cross_entropy(decoder_output, y[:, i])).sum()
                mask = mask * (y[:, i] != eos_id)
            loss = loss / valid_length
        loss.backward()

        encoder_optimizer.step(1)
        decoder_optimizer.step(1)
        decoder_init_state_optimizer.step(1)

        total_loss += loss.asscalar() / max_seq_len
        iter_times += 1
        
    if epoch % epoch_period == 0 or epoch == 1:
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = 'Time %02d:%02d:%02d' % (h, m, s)
        if epoch == 1:
            print_loss_avg = total_loss / len(data_iter)
        else:
            print_loss_avg = total_loss / epoch_period / len(data_iter)
        loss_str = 'Epoch %d, Loss %f, ' % (epoch, print_loss_avg)
        print(loss_str + time_str)
        if epoch != 1:
            total_loss = 0.0
        prev_time = cur_time

Epoch 1, Loss 0.128682, Time 00:00:20
Epoch 2, Loss 0.253116, Time 00:00:20
Epoch 3, Loss 0.123600, Time 00:00:20
Epoch 4, Loss 0.123781, Time 00:00:20
Epoch 5, Loss 0.123707, Time 00:00:20
Epoch 6, Loss 0.123252, Time 00:00:20
Epoch 7, Loss 0.123175, Time 00:00:20
Epoch 8, Loss 0.122939, Time 00:00:20
Epoch 9, Loss 0.122633, Time 00:00:20
Epoch 10, Loss 0.122371, Time 00:00:20
Epoch 11, Loss 0.122463, Time 00:00:20
Epoch 12, Loss 0.122725, Time 00:00:20
Epoch 13, Loss 0.122704, Time 00:00:20
Epoch 14, Loss 0.121717, Time 00:00:21
Epoch 15, Loss 0.121645, Time 00:00:21
Epoch 16, Loss 0.122059, Time 00:00:21
Epoch 17, Loss 0.121816, Time 00:00:21
Epoch 18, Loss 0.121900, Time 00:00:21
Epoch 19, Loss 0.121811, Time 00:00:21
Epoch 20, Loss 0.121505, Time 00:00:21
Epoch 21, Loss 0.121060, Time 00:00:21
Epoch 22, Loss 0.121095, Time 00:00:21
Epoch 23, Loss 0.121344, Time 00:00:21
Epoch 24, Loss 0.122660, Time 00:00:21
Epoch 25, Loss 0.121384, Time 00:00:21
Epoch 26, Loss 0.121117, Time 00:0

In [37]:
def test(x, y):
    nd_x = x
    x = [int(_) for _ in x.asnumpy().tolist()]
    y = [int(_) for _ in y.asnumpy().tolist()]
    print('Input: ',' '.join(input_vocab.to_tokens(x)).split('<eos>')[0])
    #print('Expect: ',' '.join(output_vocab.to_tokens(y)))
    
    encoder_state = encoder.begin_state(func=mx.nd.zeros, batch_size=1, ctx=ctx)
    encoder_outputs, encoder_state = encoder(nd_x.expand_dims(0), encoder_state)
    encoder_outputs = encoder_outputs.flatten()
    
    decoder_input = nd.array([output_vocab.token_to_idx[BOS]], ctx=ctx)
    decoder_state = decoder_init_state(encoder_state[0])
    output_tokens = []
    
    for _ in range(max_output_len):
        decoder_output, decoder_state = decoder(
            decoder_input, decoder_state, encoder_outputs)
        pred_i = int(decoder_output.argmax(axis=1).asnumpy()[0])
        if pred_i == output_vocab.token_to_idx[EOS]:
            break
        else:
            output_tokens.append(output_vocab.idx_to_token[pred_i])
        decoder_input = nd.array([pred_i], ctx=ctx)
    print('Output:', ' '.join(output_tokens))

In [17]:
def test(x):
    print('Input: ', x)
    input_tokens = x.split(' ') + [EOS]
    while len(input_tokens) < max_seq_len:
        input_tokens.append(PAD)
    inputs = nd.array(input_vocab.to_indices(input_tokens), ctx=ctx)
    encoder_state = encoder.begin_state(func=mx.nd.zeros, batch_size=1, ctx=ctx)
    encoder_outputs, encoder_state = encoder(inputs.expand_dims(0), encoder_state)
    encoder_outputs = encoder_outputs.flatten()
    
    decoder_state = decoder_init_state(encoder_state[0])
    decoder_input = nd.array([output_vocab.token_to_idx[BOS]], ctx=ctx)
    output_tokens = []
    
    for _ in range(max_output_len):
        decoder_output, decoder_state = decoder(
            decoder_input, decoder_state, encoder_outputs)
        pred_i = int(decoder_output.argmax(axis=1).asnumpy()[0])
        if pred_i == output_vocab.token_to_idx[EOS]:
            break
        else:
            output_tokens.append(output_vocab.idx_to_token[pred_i])
        decoder_input = nd.array([pred_i], ctx=ctx)
    print('Output: ', ' '.join(output_tokens))
    return ' '.join(output_tokens)

In [18]:
with open('test_data.txt','r') as f:
    test_data = f.readlines()
test_data = [data[:-1] for data in test_data]
ans = []
for data in test_data:
    ans.append(test(data))
with open('ans-none.pickle','wb') as f:
    pickle.dump(ans,f)

Input:  上 高铁 了 ， 在 最后 一 节 车厢 ， 心情 很 悲壮 ， 为什么 没有 安全带
Output:  你 ， ， ， ， ，
Input:  这 同学 真是 实在 人 啊 。
Output:  学生 ， ， ， 教育 ， ， ，
Input:  谢谢 我 心目 中 最 优秀 的 乐队 ， 我们 不 离 不弃 ！
Output:  支持 ， ， ， 超前 ， ，
Input:  尼玛 ， 建党 大爷 都 被 跌 透了 ！
Output:  你 ， 图片 ， 看上去 啊
Input:  所有 的 大学 老师们 都 应该 看看 介个 … … 真心 霸气 ！ 「 人人 网友 」
Output:  我 ， ， ， 的 觉
Input:  你 要是 嫌 你们 家 网速 慢 你 可以 调 啊 ！ 谁 拦 着 你 了 ？ 5 分钟 搞定 .
Output:  如 可爱 如 ， ， ， ，
Input:  两 年 前 的 今天 ， 36 氪 网站 上线 。
Output:  这个 ， ， ， ， ？ ？
Input:  这 张 侧脸 真 是 太 完美 了 。 ！
Output:  这 ， ， ， 出售 ， ，
Input:  不 是 我 吓唬 你 ， 北京 此时 空气 质量 差到 极致 了 （ 500 ） ， 有 人 管 吗 ？
Output:  还 ， ， ， ， ，
Input:  献给 所有 和 奶奶 有 着 童年 的 孩子 ！
Output:  伊春 ， ， 使用 ， ， ，
Input:  去 乌镇 ， 不 走 回头 桥 ， 迷醉 在 江南 水烟 中 。
Output:  我 叫做 ， ， ， ， 叫做
Input:  如果 你 曾 歌唱 黎明 ， 那么 也 请 你 拥抱 黑夜 。 （ BY 佚名 ） 早安 ， 各位 。
Output:  我 ， ， ， ， ， ，
Input:  所谓 垃圾 书 ， 难道 最 垃圾 的 不 是 教科书 吗 ？
Output:  我 的 的 的 的
Input:  谢谢 大家 喜欢 我 的 新 头像 啊 … …
Output:  班比 ， 的 sooooo cute 发展
Input:  # 小米 2S# 真容 曝光 ， 有 图 有 真相 ！
Output:  好 is ， ， 启吧 ，
Input:  【 心理 测试 

Output:  这个 同志 ， ， ， ， ，
Input:  美国 报界 评出 十 大 最 离奇 的 新闻 。 其中 一 件 新闻 是 这样 的 （ 转 ）
Output:  因为 ， ， 的 的 ，
Input:  据 微 博 消息 ， 这 是 帝都 。 请问 帝都 的 良心 -- 下水道 呢 ？
Output:  好 ， ， ， ， ， ，
Input:  被 特别 在乎 的 人 忽略 ， 会 很 难过 ， 而 装做 不 在乎 ， 会 更 难过 。
Output:  因为 ， ， 那 说到 心里
Input:  这个 年头 不 抽烟 的 人 真 难找 ， 不抽 的 请 转发 ！ （ 转 ）
Output:  这个 烟味 ， ， 习惯 。
Input:  晚霞 很 漂亮 ， 大家 周末 愉快 ！
Output:  我 ！ 承受 了 ！
Input:  奇虎 360 公开 “ 劫持 ” 媒体 ， 实在 太 过分 了 ！
Output:  这样 ， 这样 这样 这样 这样
Input:  那些 牛 B 哄哄 的 外国 法律 （ 转帖 ）
Output:  好 ， ， ， ， ，
Input:  你们 真是 够了 ！ 秀 恩爱 什么 的 最 讨厌 了 ！ （ 图 - 9gag ）
Output:  哟 是 黄小喵 ， ， 脸 淡定 脸 gj
Input:  《 愤怒 的 小鸟 》 便当 ， 可爱 哦 。
Output:  我 ， ， ， ， ， ，
Input:  Weico 微博客 户端 1.0.6 版 分组 界面 。
Output:  我 ， ， ， 的 ， 的
Input:  某些 媒体 ， 以及 某些 观众 ！ 他们 是 配套 的 。
Output:  如果 不学 会计 ， ) )
Input:  4月 21日 ， 雅安 地震 灾区 ， 微笑 的 大叔 。 （ 图 / 于鹏 ）
Output:  这 ， ？ ？ ？ ？ ？ ？ ？ ？ ？ ？ ？ ？ ？ ？ ？ ？ ？ ？
Input:  清华 图书馆 旁边 新 添 的 雕塑 ， 雷 死人 不 偿命 吗 请问 ？
Output:  这样 ， ， ， 这样 ，
Input:  # 轻松 一 刻 # 苹果 三 星 专利 战 “ 真相 ”
Output:  研究 ， 的 的 的 的
Input:  其实 篮球 运

Output:  我 ， ， ， ， ，
Input:  这 尼玛 天气 也 太 热 了 。
Output:  这个 ， ， ， ， ，
Input:  愚人节 最强 整人 活动 ， 后排 的 老 湿 杯 具 鸟 。
Output:  哈哈 ， ， ， ， 。
Input:  唉 。 才 发现 我 简直 是 一无是处 。  中 枪 的 。 默默 转
Output:  好 ， ， ， ， 讲述 二人转
Input:  Roger Federer 输 了 ( 。 _ 。 ; ) 我 的 心碎 了 ！
Output:  以后 ， ， ， 。 。
Input:  谁 说 中国 导演 拍 不 出 大 场面 ， 我 跟 他 急 ！
Output:  这 意思 是 的 的 ， 拆
Input:  尼玛 。 下次 剪毛 先 征求 下 我 的 意见 好不 ？
Output:  这 的 的 的 的 。
Input:  我 过 不过 光棍 节 无所谓 ， 只要 我 喜欢 的 人 也 过便 好 了 。 （ 转 ）
Output:  这 ， 又 ， ， ， ，
Input:  湖南 凤凰 ， 古镇 之 夜 。 你 心动 了 吗 ？
Output:  我 ， ， ， ， ， ，
Input:  “ 我 恳请 童鞋们 封杀 我 。  ” 一一 杜甫 在 接受 采访 时 如是 说 。
Output:  我 ， ， ， ， ，
Input:  这 位 童鞋 。 你 尊 的 好 霸气 ！ 就 你 最 霸气 ！  「 转 」
Output:  给 ， ， ， ， ，
Input:  # PPT 演讲 技巧 # 如何 设置 ppt ， 使 讲演者 看到 备注 ， 而 观众 看 不 到 ！
Output:  这个 已经 的 的 的 希瑞
Input:  英国 ： 全民 “ 减盐 ” 摄 盐 量 每 日 6 克 alink
Output:  想 想 想 ， 的 动画片
Input:  楞装 纯情 小 少女 啊 哈哈哈
Output:  哈哈 ， ， ， 打 ，
Input:  婊子 君 ， 四川 的 自助餐 是 不 是 就是 吃 各 种 辣椒 ？
Output:  这 时候 是 的 的 时候
Input:  写 了 一 段 非常 温暖 的 家庭 戏 ， 心里 暖 洋洋 的 。 真 美好 ！
Output:  

Output:  这 ， ， ， 的 ， 的
Input:  这 是 多么 严苛 的 人生 啊 … … 吓 尿 了 ！
Output:  尼玛 不 不 ， 不 了
Input:  一百 块 的 忧伤 ！ 最后 知道 真相 的 我 眼泪 掉下来 啦 ！
Output:  哈哈 ， ， ， 的 。
Input:  在 一 台 如此 腐的 晚会 上 高唱 中国 范 就 是 这么 帅 真是 太 嚣张 了
Output:  这 让 不 的 的 的 的
Input:  看完 这 图 你 还 敢说 学过 美术 吗 ？  「 转 」
Output:  好好 休息 创意 ， ， ， ，
Input:  绝对 ， 绝对 不 能 再 胖了 ！
Output:  美国佬 ， ， ， 的 ，
Input:  给 穷 逼 的 七夕 の礼 ， 特别 篇 《 穷 逼 就 别 过节 了 》
Output:  想 ， ， 个 ， ，
Input:  【 两 性 心理 】 好 女人 疼 男人 的 十 个 小 细节 。
Output:  这个 ， ， ， ， ， ，
Input:  最好 用 最 贴心 的 iPhone 支架 在 这里 。
Output:  荒岛 余生 汤姆 汉克斯 的
Input:  xx 到此一游 的 症结 其 实在 《 西游记 》 ， 我 相信 很多 人 都 是 从这 学的 ！
Output:  我 ， ， ， ， ， ，
Input:  很 喜欢 这 件 衣服耶 。 谢谢 啦 。
Output:  我 ， ， ， ， ，
Input:  金融 时报 ： 国 开行 将向 重庆 提供 新 资金 支持 。
Output:  这个 ， ， ， ， ，
Input:  只 需 7 步 ， 任何 魔方 6 面 皆 可 还原 。 喜欢 的 童鞋 收藏 啦 。
Output:  洗脑 ， 待客 ， ， ， ，
Input:  【 中 泰 对话 神贴 】 看完 后 有 哭的 、 有笑 的 、 有 沉默不语 的 ?
Output:  这 ， ， ， ， ，
Input:  关于 石家庄 、 南阳 等 “ 海底 捞 ” 非我 公司 店面 的 声明 alink
Output:  这 ， ， ， ， ，
Input:  再见 ， 小贝 ， 致流 走的 青春 。 祝福 ！ 晚安 ！
Output:  我 vs

Output:  这 ， ， ， ， ，
Input:  乌克兰 某个 火车 隧道 ， 好 清新 好 浪漫 ！
Output:  这 可爱 多 的 的
Input:  五 磅 脂肪 五 磅 肌肉 对比
Output:  你 ， 的 喜欢 喜欢 ！
Input:  日本队 提出 了 抗议 ， 认为 对 Kohei 的 鞍马 判分 不公 。
Output:  哈哈 ， ， ， 的 。
Input:  这 就 是 高帅富 和 你 的 区别 ， 不服 来辩 ！
Output:  我 的 的 的 的 的 的
Input:  可爱 的 美妹 带给 我 很多 欢笑
Output:  这 ， 的 的 的 的 的
Input:  为什么 选择 喜剧 行业 ， 这 理由 太 经典 了 。 「 蛋花 儿网 」
Output:  哇 ， ， ， ， ， ，
Input:  有 人 说 看见 我 在 做 法事 。
Output:  我 ， ， ， ， ，
Input:  据说 ， 这 是 最 让 艺术家 讨厌 的 问题 。 是 这样 么 。
Output:  瞧 * 单位 ， ， 年终 ， 还木
Input:  转 ： 广州 有 这么 奇妙 的 商场 ， 屋顶 很 美 。 没有 顾客 ， 很 安静 ： ）
Output:  这个 ， 能 能 能 能
Input:  我的 室友 是 个 贱人 ！ 就 这样 被 欺骗 了 ！ 被 欺骗 了 ！ （ 转 ）
Output:  因为 ， ， 黑色 ， 黑色 ，
Input:  这 首 情诗 太 棒 了 。 嘻嘻嘻 嘻嘻 （ via 货 篮子 ）
Output:  我 ， ， ， ， ， ， ，
Input:  其实 我 是 晚上 七 点 拍 的 ， 但 我 还是 喜欢 这会儿 发 …
Output:  我们 ， ， ， 烧 ， 的
Input:  按 最新 的 舆论 口径 ， 这儿 得 算 罗马 大学 神学院 了吧 。
Output:  垃圾 魔都 ， 套 ， 。
Input:  长枪 大炮 ， 相机 快门 的 残酷 战争 ！ alink
Output:  我 的 的 的 的 的
Input:  【 午餐 】 这 点 够 谁 吃 ！ 这 点 够 谁 吃 ！ 这 点 够 谁 吃 ！
Output:  老 ， ， ， ， ，
Input:  冲 超 了 

...

Output:  我 ， ， ， 不过 ， 不过
Input:  速 来 膜拜 本年度 最 佳 性感 男 神 露 肉照 ！
Output:  这 配音 的 的 渣啊 的 的
Input:  小时候 这样 快乐 过 ， 帅 过 的 童鞋 ， 请 举 下手 ！ （ 图 转 ）
Output:  额 ， ， ， ， ！
Input:  你 好 ， 九月 ！ 希望 这个 月 过好点 ！
Output:  我 ， ， 蜀黍 的 ，
Input:  这个 小子 够 懒 的 ， 下了 飞机 ， 还 要 坐在 行李箱 上面 让 别人 拖着 走 。
Output:  我 ， ， ， ， ， ， ，
Input:  天鹅 城堡 —— 德国 南部 的 地区 。 美 得 像 童话 ！
Output:  哈哈 ， ， ， ， ，
Input:  任志强 骂人 vs 孔庆东 骂人 ， 媒体 如何 评价 ， 大家 随意 感受 一下 吧 。
Output:  这 ， ， 肚子 ， ，
Input:  其实 李安 说 的 很 对 ， 详情 不 解释 。 省得 你 麻痹 又 说 我 黑 谁 。
Output:  我 是 四 的 的 ， 的
Input:  各 种 垃圾 短信 ， 各 种 莫名 电话 。 难道 你 穷 疯 了 ？ 惹得 老子 开骂 。
Output:  Yes ， 的 tomorrow 的 的
Input:  新疆 伊犁 薰衣草 田 ， 这般 美景 不比 普罗旺斯 差 。
Output:  我 ， 的 的 的 的 的
Input:  男人 情敌 大 黄瓜 ， 女人 情敌 大 菠萝 。
Output:  烟草 起来 起来 ， 尺骨 。
Input:  做 父亲 最 幸福 的 时刻 ： 儿子 做 了 你 的 一 尊 头像 。
Output:  这 ， ， ， ， ， ，
Input:  # 冷 犯罪 # 嘿 ！ 你 这样 的 也 来 抢劫 了 ？ （ via Reddit 用户 OnionPotato ）
Output:  这个 ， ， ， ， ，
Input:  这 就 是 恶意 划破 别人 汽车 轮胎 的 后果 。
Output:  坐下 ， ······会不会······坍塌 ， ， ，
Input:  《 莫言 最 伤人 的 83 句 话 》 ， 非常 经典 。
Output:  玩

Output:  我 的 的 的 的 的 的
Input:  参观 了 北京 现代 工厂 ， 试 驾 了 第八 代 索纳塔 ， 提速 之 快 印象 深刻 。
Output:  因为 ， ， ， ， ？ ？
Input:  帮主 的 开场 问题 相当 的 犀利 ！ 猜猜 是 什么
Output:  哈哈 ， ， ， ， ，
Input:  十二 星座 知道 女友 怀孕 了 什么 表现 。
Output:  我 ， ， ， ， ，
Input:  我 说 个 标准 ： 凡是 现在 去扒 赵红霞 的 媒体 ， 全 是 垃圾 媒体 。
Output:  我 ， ， ， ， ，
Input:  今天 ， 无缘无故 ， 心情 很 好 ， 多谢 大家 ！
Output:  这 发誓 填词 一定 的 超越 黄伟文 林夕 林夕
Input:  有 群众 说 这 是 口水 烘干机 。 可 你们 觉得 有用 嘛 。
Output:  真 是 但 但 但 但 但 但 但 但 但 但 但 但 但 但 但 但 但 但
Input:  小鸟们 如果 有手 ， 好 有 范儿 ！ 「 转 」
Output:  全 ， ， ， ， … … … … … … … … … … … … … … …
Input:  这样 洗手 洗 得 干净 吗 ？ ！ ？ ( °_° )
Output:  我 750 mb 流量 猪 ， 重置
Input:  世间 最 痛苦 的 莫过 于此
Output:  喵星人 ， ， ， ， ，
Input:  膜拜 一下 巴 神 的 成名作 吧
Output:  好靓 人物 ， ， ， ！
Input:  可爱 的 熊猫 蛋糕 ， 还是 抹 茶味 来 的 。
Output:  曾经 ， ， ， ， ， 的 的
Input:  IPO 新政 ， 也许 应该 出 点 利好 来 中和 中和 。
Output:  这 的 的 的 的 。
Input:  哈士奇 与 狼 的 区别 ！ 「 转 」
Output:  我 ， ， ， ， ，
Input:  弯腰 也 不 影响 改变 世界 。
Output:  这个 ， 的 ， 的 好
Input:  告诫 还没 结婚 的 男 同胞们 ： 母乳 量 ， 和 胸 大 胸 小 没关系 。
Output:  这 ， ， ， 的 人
Input:  原来 吸

Output:  我 ， ， ， ， ， ， ，
Input:  从 今天 直接 请假 到 国庆节 不 被 老板 骂 ， 就 靠 这个 了 ， 你 信 不 信 ？
Output:  哈哈 ， ， ， ， ，
Input:  在 这样 的 冬天 啊 ， 我 像 一 只 河豚 一样 憋屈 忧伤 。
Output:  不 不 不 不 不 不 不
Input:  你 信 不 信 我 打 你 爹 ！
Output:  哎耀 ， 杨澜 前辈 国字 的 时
Input:  农夫 求助 ： 兔子 养 这么 大 之后 该 如何 处理 。
Output:  村医 班 的 妹子 的 的
Input:  这 鸡肉 肥 而 紧实 ， 有 嚼劲 ， 太 赞 啦 ！ 我 在 这里 : alink
Output:  不 威斯敏斯特宫 是 ， 的 ，
Input:  爱 拼 才 会 赢 ， 不 吃 才 会 瘦 ！ 送给 所有 在 春天 有 梦想 的 人 ！
Output:  哇 得 ， ， ， 得
Input:  “ 我 恳请 童鞋们 封杀 我 。 ” 一一 杜甫 在 接受 采访 时 如是 说 。
Output:  顺应 潮流 ， 旅行 ， ，
Input:  平壤 的 爱情 ， 祝福 中秋节 快乐 。
Output:  哈哈 的 的 的 的 的
Input:  故意 歪头卖萌 神马 的 很 可耻 啊 。
Output:  哈哈 ， ， ， ， ，
Input:  希望 的 是 ： 今晚 阿根廷 能嬴 … 简单 就 是 快乐 ！
Output:  我 ， 狐狸 的 的 的
Input:  这 钟点 怎么 忽然间 有点 饿 呢 ？ 看 着 相片 能 望梅止渴 吗 ？
Output:  你 你 你 ， 你 你 谢天谢地
Input:  麦当劳 全 国 通用 的 无线 上网 密码 。
Output:  这 ， ， … … … …
Input:  这样 的 安全套 商家 真 无耻 ！
Output:  这 篇 ， ， 烦人 很
Input:  这 才 是 新闻 ， 这个 人 到底 是 谁 呢 ？ 下次 还 能 进京 吗 ？
Output:  这 的 的 的 的 的
Input:  “ 你 最近 还 好吗 ？ ” “恩 。 ” “ 你 在 骗 我 吗 ？ ” “恩 。 ”
Output:  还 · 吃 吃 ·

Output:  因为 ， ， ， ， ，
Input:  北美 白玉兰 、 城市 噪音 、 纽约 晨 。
Output:  又 ， ， ， ， ， ，
Input:  # Google I/O 2013# 今晚 的 演讲 环节 结束 了 。 祝 大家 好 梦 。 （ 图 via TheVerge ）
Output:  说的 ， ， 楼 的 的 的 的
Input:  天冷 了 ， 请 为 同伴 添衣 。 太 感人 了 有 木 有 。
Output:  我 ， 的 的 的 。
Input:  爸爸 请 不要 剪掉 我 的 翅膀 ， 我 要 像 何 北 一样 自由 飞翔 ！
Output:  给 ， ， ， ， ，
Input:  蓝莓 布丁 蛋糕 。 好 美 的 蓝 。 喜欢 的 速度 转吧 ！
Output:  等 ， ， ， 的 =
Input:  那些 把 图片 / 答案 放在 相册 里 骗 关注 的 人 ， 最最 最 恶心 了 ！
Output:  这 是 的 的 的 的 的 的
Input:  浙江 卫视 《 中国 好 声音 》 ， 好看 ， 记住 你 了 。
Output:  我 ， ， ， ， ， ， 了
Input:  酒足饭饱 ， 晚安 我 爱 的 人们 ， 晚安 香格里拉 。
Output:  我 ， ， ， ， ， ，
Input:  参加 一 个 朋友 的 婚礼 ， 有 茅台 ， 开车 ， 不喝 ， 爱 谁谁 。
Output:  阴霾 年龄 ， ， ， ，
Input:  其实 大家 都 喜欢 谎言 ， 大家 讨厌 的 只 不过 是 ， 自己 被 谎言 骗 了 。
Output:  曾经 ， ， 自私 真 真 ，
Input:  南京 太 热情 了 吧 ！ 大家 小心 ， 不要 弄到
Output:  我 ， ， ， 的 ，
Input:  世界杯 真 讨厌 ， 欧 冠 也好 讨厌 … … 点球 神马 的 最 讨厌 了 。
Output:  不要 ， ， ， ， ， ，
Input:  情人节 ， 穿 新 毛衣 的 米兔 ， 祝 所有 米粉 爱情 甜甜蜜蜜 ！
Output:  这 ， 都 的 的 的
Input:  瞧这 一 家子 ， 小 M 虽然 是 个 新人 ， 但 娘家 的 火力 十足 啊 。
Output:  晚上 ， 的 的 的

Output:  这 ， ， ， ， ？ ？ ？ ？
Input:  # 无助 情话 # 落幕 听起来 好过 散场 ， 就 好比 安静 好过 凄凉 。
Output:  来 ， ， ， 来 ，
Input:  哈哈 、 主人 我 给 你 捡 回来 了 ！ 主人 我 给 你 捡 回来 了 ！ > 转 <
Output:  向 ， 致敬 ， ， ，
Input:  在 地中海 边 赤身 二 日 读 《 西藏 生死 之 书 》
Output:  这 ， ， ， 的 ，
Input:  希望 有 这样 一 个 坐骑 ！ 省 油 、 省钱
Output:  完全 的 的 ？ ？ ？ ？
Input:  还有 谁 记得 当年 的 暴力 摩托 ， 童年 的 回忆 。 （ 转 ）
Output:  羡慕 ， ， 旅行 的 ，
Input:  现在 的 小学生 ， 伤 不 起 啊 伤 不 起 ！ 笑死 我 了 。
Output:  这 是 是 是 的 ，
Input:  女生 最 讨厌 男生 几 大 装扮 ， 你 中 了 几 枪 ？ （ 转 ）
Output:  混 在一起 傻逼 傻逼 才 傻逼 傻逼 傻逼 傻逼
Input:  超 可爱 的 三胞胎 ， 一眼 就 能 看出 谁 是 老大 老二 老三  「 转 」
Output:  我 ， ， ， ， ，
Input:  能 把 一 只 鸡 （ 鸭 ） 摆成 一 头 牛 的 人 ， 功力 很 大 ， 很 深厚 。
Output:  这 ， ， ， ， ，
Input:  这 哥们 几乎 没有 不会 的 项目 ， 他 就是 我 的 男 神 —— 普京 。 ( 转 )
Output:  为什么 ， ， 的 的 ，
Input:  我 女 朋友 老 想要 偷拍 我  「 转 」  PS ： 可爱 的 右手
Output:  这 ， 的 的 的 ， 的 的
Input:  老师 罚 他抄 一千 遍 单词 。 可是 。 老师 看后 疯了 ！ 淫 才 啊 ！
Output:  我 ， ， ， ， ， ， ，
Input:  很多 时候 ， 钱 是 富人 的 手段 、 穷人 的 诱饵 。
Output:  闲话 ， ， 是 ， 。
Input:  发现 一 颗 精子 翱翔 在 蓝天 上 。
Output:  哈哈哈 ， 反复 ， ， ，
Input

In [38]:
for i in range(0,2890):
    test(X[i],Y[i])
    print('\n')

Input:  中国 移动 营销 行来 发展 报告 alink 
Output: 王 大姐 ， 打字 细心 一点


Input:  小马 也 疯狂 ------ 地位 之 争 。 
Output: 于 老师 他们 告诉 劝劝 架么


Input:  那些 年 ， 我们 一起 偷看 过 的 电视 。 「 暴 走 漫画 」 
Output: 真不愧是 的 走 的 走 爹······· 的


Input:  北京 的 小 纯洁们 ， 周日见 。 # 硬汉 摆 拍 清纯 照 # 
Output: 嗷嗷 ， 湿的 左手 干嘛 干嘛 ，


Input:  要是 这 一 年 哭泣 的 理由 不 再 是 难过 而 是 感动 会 多么 好 
Output: 那 快 快 感动 快 哭了


Input:  对于 国内 动漫 画作者 引用 工笔 素材 的 一些 个人 意见 。 
Output: ( ﾉ´∀ ｀ * ) ﾉ 大师


Input:  猫咪 保镖 最 赞 了 ！ 你们 看懂 了 吗 ？ ！ （ 来自 ： 9gag ） 
Output: 喵喵 保镖 抢镜了哟 ， 幸亏 ，


Input:  你 见 过 皮卡丘 喝水 的 样子 吗 ？ 
Output: 你 东西 ， 东西 东西 东西 都烦


Input:  如果 有 个 人 能 让 你 忘掉 过去 ， 那TA 很 可能 就是 你 的 未来 。 
Output: ····关键 我 那人 是否 忘记 他 过去 过去


Input:  我 在 北京 ， 24 岁 ， 想 去 马尔代夫 ， 一 个 人 。 
Output: 我 ， 太原 我 24 岁 我 都 捷克 捷克


Input:  哥 你 还 跳 不 跳楼 了 ？ 我们 要 下班 啊 ！ 
Output: 其实 想 想 很多 很多 想 的 其实


Input:  龙 生 龙 ， 凤 生 凤 ， 是 个 喵咪 它 就萌 。 
Output: 微信都 加了 ， 说好 ， 8.20


Input:  从 胚胎 期 开始 的 面部 特征 演变 过程 
Output: 自然 生理 变化 变化 ， 面部 演化 过程


Input:  本 届 轮值 主席 王石致 开幕词 。 讲 60 岁 上 哈佛 。 
Output: 任总 带 ， ， ， 机会


Input:  非常

Output: 必须 ， 俺家 老刘 ， ， ， ， 情人


Input:  赶路 ， 司机 给 我 补课 ， 给 我 听 吴莫愁 的 歌 。 
Output: 吴莫愁 ， 欣赏 欣赏 欣赏 ， 敢于 发泄 灵魂


Input:  火车站 和 长途 汽车站 才 是 当今 中国 ， 飞机场 不算 。 
Output: 同意 的 总是 的 外表 奢华 内脏 糜烂


Input:  看到 最后 的 文字 ， 终于 懂 了 ！ 
Output: 这 社会 现象 电话 异地恋 电话 XXOO 维护 感情


Input:  #UC9 语录#不要 害怕 批评 的 声音 ， 因为 那 是 对 你 最好 的 鞭策 。 
Output: iPhone 手机 ， 最新 手机 版本 信息 信息 登陆 信息 信息 信息 信息 信息 信息 信息 信息 信息 信息 信息


Input:  告诉 你 ！ 哥抱 的 不 是 美女 ， 是 旺财 ！ 
Output: 我 比较 关心 狗 关心 狗 狗 狗 狗 狗 性什么 性什么 性什么 性什么 性什么 性什么 性什么 性什么 性什么 性什么


Input:  世界 银行 行长 已 面临 非常 严峻 的 失业 危机 
Output: 真实 真的 的 煮 的 的


Input:  兰州 牛肉面 的 江湖 风云 榜 （ Via : 茉莉 Sunmoly ） 
Output: 我 ， 行 ' ' ' ' ' Forever Forever


Input:  与 NBA 球员 比手大 ！ 猜猜 哪 只 手 是 我 的 ？ 
Output: 胡说 ， 只 才是 ， ， 才是


Input:  小时候 吃 过 这 朵 花 的 请 举手 ！ 
Output: 这 吃法 的 已 传遍 大江 南北


Input:  某 人 已经 彻底 废 了 ， 她 的 新歌 以后 不 听了 
Output: 去年 今年 差别 ， ， ， 莫非 去年 假唱 假唱


Input:  绝望 了 ！ 对 这个 卖萌 的 世界 绝望 了 ！ 
Output: 兔斯基 ， 撑 ， 住 住


Input:  # 每 日 趣 图 # 这 炫 技 的 停车 ， 太 拉 风 了 ！ 
Output: 哪里 人 人 人 不是 人


Input:  复古 - 惊艳 ！ 真心 太 美了

Output: 喜欢 静姐 身上 黑色 毛衣 黑色 告诉 告诉 告诉 牌子 牌子 牌子 牌子 牌子 牌子 牌子 牌子 牌子 牌子 牌子


Input:  沉默 ， 可以 让 混乱 的 心 ， 变得 清澈 。 
Output: 沉默 心 心 让 混乱 ， 心 心 变得 清澈 清澈 清澈 清澈 清澈 清澈 清澈 清澈 清澈 清澈 清澈


Input:  老 爵爷 ， 向 您 致敬 ！ 没有 您 ， 就 没有 曼联 ！ 
Output: 谢谢 ， 爵爷 ， 谢谢 曼联 20 20 冠


Input:  美丽 镜头 下 的 那 份 宁静 ， 让 你 彻底 爱上 这里 —— 西塘 
Output: 去过 ， 人少 ， 的 很美 时候 时候 时候


Input:  据说 这 是 女士们 风险 低 、 回报率 最 高 的 一 种 投资 方式 ！ （ 转 ） 
Output: 月薪 还是 年薪 还是 日元 还是 还是


Input:  大 S和富 二 代 结婚 之后 就 变成 了 大 $ 
Output: 大 S和富 二 结婚 结婚 变成 变成 变成


Input:  「 请便 后冲 便 以便 后 便 者 再 便 」 ， 求 下联 。 
Output: 求 种 种 留 留 种 种 种 种 种 种 种 种 种 种 种 种 种 种 种


Input:  化妆 ing ， 一会儿 录影 开始 ， 18 期 ， 加油 … 
Output: 噗 ， ， 故意 ， ， ， Auv Auv 楠楠姐 楠楠姐 楠楠姐 楠楠姐 楠楠姐 楠楠姐 楠楠姐 楠楠姐 楠楠姐 楠楠姐 楠楠姐


Input:  #S+# 这个 点 大家 都 在看 # 中国 好 声音 # ？ 
Output: 我 ， ， 来着 但是 ， 逛 逛 超市


Input:  ★ 世界上 10 家 最美 的 书店 。 
Output: 等我 ， ， ， ， ， ，


Input:  男团 半决赛 两 个 功臣 
Output: 娃姐 给做啥 好吃 的 的 的 给乐乐乐成


Input:  “ 诺 不 轻信 ， 故人 不 负 我 ， 诺不轻许 ， 故我 不 负 人 。 ” 
Output: 诺不轻许 ， 故我 ， 负 ，


Input:  长颈鹿 长颈鹿 。 乃 滴 脖纸 为什么 介么 长 。 
Output: 小胖

Output: 输赢 的 死 的 的 事 事


Input:  白塔山 背后 ， 好像 正在 拆迁 。 
Output: 为 迎接 ， ， 到来 而 拆迁 而


Input:  【 心灵 探索 】 不 以 结婚 为 目地 的 谈恋爱 ， 都 是 在 替 别人 养老婆 。 
Output: 不以 结婚 为 目地 谈恋爱 谈恋爱 ，


Input:  以 我 的 性格 来说 ， 其实 一定 宁可 两 边 是 不 对称 的 ！ 
Output: 就是 就是 ， ， 特别是 ， ， 不对称


Input:  看看 亚马逊 和 申通 在 忙碌 的 时候 的 区别 。  （ 转 ） 
Output: 管理 是 是 送到 啊 买家 手里


Input:  【 一 张 图 告诉 你 守门员 曾 诚有 多 勇猛 ！ 】 丨 来自 : 央视 新闻 
Output: 必须 转 开 昨天 ， 开 棒子 打平 开 离不 开 开 操作 操作 操作 操作 操作 操作 操作 操作


Input:  【 求证 】 这 就 是 撕毁 “ 西哈努克 画像 ” 的 那个 中国 女子 吗 ？ 
Output: 致敬 ， 外交部 ， 哎 哎 哎 哎 哎 哎 哎 哎 哎 哎 哎 哎 哎 哎 哎 哎


Input:  一 个 从 互联网 行业 跳槽 到 公务员 的 朋友 写的 ， 写 的 非常 受用 。 
Output: 第一 次 见到 缩略 图 图 图 图 整齐 整齐 文章 文章 文章 文章 文章 文章 文章 文章 文章 文章


Input:  训练有素 。 那个 ， 那个 第三 位 童鞋 ， 麻烦 把 你 的 狗腿 拿开 ！ 
Output: 狗腿 是 是 是 狗腿 ！


Input:  见 朋友 家用 过 季 的 菜苔 作 插花 ， 小 黄花 雅致 ， 挺 美 。 
Output: 我 中 瓶子 透明 的 瓶子 瓶子 瓶子 养大蒜


Input:  希望 大家 帮忙 转发 人肉 昨晚 偷走 了 我 的 笔记本 谢谢 大家 
Output: 找到 直接 弄死···妈勒··· ， 遇到 ，


Input:  死 前 一定 要 破处 ！ 不然 上面 会 有 恐怖 分子 等着 你 啊 ！ —— 教堂 标语 
Output: 哈哈哈 ， 72 72 处女 哪 比得 哪 72


Input:  

Output: 我 ， 家 ， ， 生物 家


Input:  普通 狗 ， 文艺狗 ， 和 2 B 狗 。 
Output: 哈哈 最后 张 ， 。 。


Input:  每个 人 ， 都 希望 成为 另 一 个 人 ， 或 被 当成 另 一 个 人 。 
Output: 哦 ， 的 需要 需要 不堪


Input:  痘痘 告诉 你 ， 身体 哪里 生病 了 ！ （ 转 ） 
Output: 压力 ， 脾气 差 ··· ··· ··· ··· ··· ··· ··· ··· ··· ··· ··· ··· ··· ··· ··· ···


Input:  仔细 想想 ， 人 活着 其实 就是 在 销售 ， 永远 在 销售 自己 ！ 不 是 吗 ？ 
Output: 洗脑 学 成功 学 成功 学 成功


Input:  做 手机 的 应该 选择 android ， 做 移动 互联网 的 ， 应该 毫不 犹豫 的 选择 iOS 
Output: 我 鞠躬 二 一 鞠躬 一 一 鞠躬 拜 拜 拜 拜 拜 拜 拜 拜 拜 拜 拜 拜


Input:  国足 比赛 凌晨 1点 ， 现在 才 10点半 ， 怎么 捱呢 ？ 
Output: 我 ， 我 想说 聊 ， 毛 钱 钱 五


Input:  # NBA 酷 图 # 一转眼 ， 雷 已 近 不惑 
Output: 总决赛 第六 那个 那个 价值 千金 那个 三分


Input:  第一 天 适应 比赛 场地 ， 感觉 不错 ！ 我 在 : alink 
Output: 游泳 池水 的 干净 的 。


Input:  微信 ， 历时 433 天 ， 从 0 到 100，000，000 用户 ！ weixin . qq.com alink 
Output: 强烈 要求 微信出 Meego 版 了


Input:  第一 次 做 寿司 ， 果然 很 容易 
Output: 想起 ， 以前 以前 大学 宿舍


Input:  一 针 见血 的 品牌 定位 ， 各位 骚年 赶紧 对号 入座 … … 「 转 」 
Output: 一不小心 我 成 失足 青年 我


Input:  缘 来 要 惜 ， 缘尽 要 放 ， 随方 就 圆 ， 无处 不 自在 。 
Output: 缘 ， ， 惜 ， 缘尽 放 放


...

Output: 新年 快乐 ， 恭喜发财 ， ， ， 新年 新年


Input:  居居居居 居然 下班 了 ？ 狂喜 ！ 猛收 ！ 暴走 ！ 怒睡 ！ 
Output: 宝哥 ， 听说 能 能 而且 网管


Input:  老 了 ， 讲课 站 三 个 小时 ， 腰疼 。 
Output: 咋 老师 给 老师 搬 椅子 椅子 老师


Input:  日本 最新 创意 红绿灯 。 ！ 
Output: 我 帅气 ， ， ， ， 大大 红灯 闯


Input:  上 了 N 年 语文 课 的 孩纸 谁 都 伤 不起 。 
Output: 尼玛 早 ， ， ， 说过 人


Input:  那些 年 ， 那些个 使 人 难忘 而 又 充满 辛酸 的 动人 故事 。 
Output: 补上 羊 与 狗 与 风筝 原图 原图


Input:  # 午间 分享 # 每 一 部 大片 的 海报 背后 都 有 一 个 宿敌 。 
Output: 难怪 “ 山寨 “ “ 英文 翻译 “ “ ” ” ” ” ” ” ” ” ” ” ”


Input:  那些 年 ， 未来 遥远 得 没有 形状 ， 我们 单纯 得 没有 烦恼 。 
Output: 那些 年 ， 未来 遥远 年 ， 形状 形状 得 烦恼 烦恼 烦恼 烦恼 烦恼 烦恼 烦恼 烦恼 烦恼 烦恼


Input:  看 一 次 笑 一 次 。 现在 男女 交往 路线图 。 
Output: 困了 ， 看 ， 笑了 ， 精神 精神


Input:  翻到 这 张 照片 ， 想 定做 一 件 旗袍 ， 不 知道 北京 哪里 比较 好 ？ 
Output: 从 回复 来看 ， 的 火药 的 真人 比她 发言 发言 发言 发言 发言 发言 发言 发言 发言 发言 发言


Input:  世界上 有些 离别 ， 是 真的 离别 。 有些 再见 ， 是 再 也 不见 。 
Output: 父亲 走得 那样 突然 ， ， 连 告别


Input:  一 张 图 告诉 你 为什么 大陆剧 找 不 到 感觉 。 
Output: イタズラなkiss なぁ ， ははははは ははははは ， ，


Input:  0805 话题 ： 你 有 没 有 动手 制作 或 改装 过 什么 东西 ， 省钱 且 创意 无限 
Output:

Output: 鄙视 得奖 都 立刻 拉黑 了 校友


Input:  巴楚 恐怖 事件 令 人 发指 alink 
Output: 致敬 ， 尊重 的 的 的 生命


Input:  小丸子 的 包装 ， 国内 有 就 好了 
Output: 什么 时候 ， 时候 ， 时候 时候


Input:  用 一 个 地名 证明 ， 你 在 杭州 生活 过 。 我 先来 ： 卖 鱼 桥 
Output: 白泽弄 ， ， ， 杭州 杭州 ，


Input:  长 款 连 衣裙 让 人 想 开始 一 场 悠闲 的 约会 ！ alink 
Output: 好 得 左上 ， 裙子 ，


Input:  中华 民国 总统府 网站 ： alink （ 免 翻 墙 ） 。 
Output: 看看 总统 每 ， 日 行程 日


Input:  易中天 真身 出没 ， 谁 与 争锋 
Output: 易中天 老师 真身 出没 的 ， 张伟


Input:  《 回忆 我 的 几 个 极品 外教 》 
Output: 臭 脚 ， 市场 市场 ， ，


Input:  这样 穿 衣服 ， 你 学 不 会 的 … … 
Output: 我 穿 衣服 学 衣服 学


Input:  疯狂 电视台 。 明天 又 加演 四 天 六 场 ， 大家 辛苦 开心 的 彩排 啊 。 
Output: 赵哥 ， ， ， ， 大学 毕业 毕业


Input:  为什么 我 觉得 蚊子 是 一 种 特别 SB 的 生物 
Output: 哈哈 就是 就是 就是 就是 傻傻的


Input:  我家 的 巧克力 你们 稀饭 吗 ？ 
Output: 我的狗狗 叫 叫 巧克力 叫 叫 腊肠


Input:  中国 的 俗人 太 多 了 。 一 个 配得上 我 的 都 没有 … … 
Output: 同意 ， 舟舟 ， ， 证据 牵强 牵强


Input:  忽然 想念 呼和浩特 老 绥远 烧麦 。 
Output: 还有 焙子 好吃 好吃 好吃 好吃


Input:  公司 安静 了 ， 自己 写字 玩 吧 。 
Output: 我 ， ， ， ， 毛笔 字


Input:  我 有 一 个 很 棒 的 妈妈 ， 我 爱 她 。 
Output: i love my mom soooooo

Output: Google Reader will not be available after July 1，2013


Input:  # 一 句 话 暴露 年龄 # 你 小学 时候 最 火 的 动画片 是 什么 ？ 晚安 ！ 
Output: 还有 多少 多少 记得 记得 希瑞 》 记得


Input:  京东 十 年 庆 3 D 生日 蛋糕 精彩 制作 过程 放送 啦 ！ alink 
Output: 确实 ， 创意 创意 祝福 祝福 祝福


Input:  有的 时候 站在 低处 往上 看 和 站在 高处 往下 看 ， 看到 的 景象 也 差不多 
Output: 飞哥 ， 跑路 ， ， ！ 羡慕


Input:  有 洁癖 、 有 生活 态度 、 有 原则 的喵 ！ 
Output: 哈哈 想起 ， ， ， ， ， ，


Input:  喜欢 家家 户户 在 阳台 上 欣赏 日落 的 感觉 alink 
Output: 如今 总是 日落 总是 伴随 总是 烟雾 缭绕 的


Input:  有 件 事 我 纠结 到 了 今天 ， 你 是否 也 有 同感 ？ 
Output: 这 应该 ， ， 商机 插头 ， 兼容 插头


Input:  不要 羡慕 你 没有 的 ， 却 忽略 你 拥有 的 。 早安 。 
Output: 我 ， ， ， 贪心 ， 拥有 ，


Input:  最 适合 南佳也 和 惠比寿 麝香 葡萄 的 是 ： 董永 与 七 仙女 。 
Output: 服了 ， 这 ， ， ， yy yy


Input:  泰国 水灾 中 的 人民 智慧 。 最后 一 张 亮 瞎了 ！ 
Output: 最后 是 是 是 传说 逼 不过


Input:  # 同花顺# 忆前 公司 两 三 事 （ 同花顺 ） alink 
Output: 这样 员工 ， 走了 走了 同花顺 完蛋


Input:  将要 飞出 雪 的 世界 。 大 深山 中 居然 密密 麻麻 的 散布 着 大量 的 村庄 。 
Output: 飞着 是 可以 发微 博么 还


Input:  < 私 たちはそれを 水で游ぶ >  （ 翻译 ： 偶们 一起 来 玩水 吧 。 ） 
Output: 这个 表情 ， 秒 杀 ， ，


Input:  要是 开放 去 钓鱼

Output: 巨蟹 男 准 的 闷骚 碗里 着 着 碗里 着 着


Input:  国梁 大校 ， 离 将军 不 远 了 。 
Output: 奥运会 ， 完 ， 大校 ，


Input:  休息 时 ， 可以 聚焦 ！ 拍 的 还 行吗 
Output: 和 ， 走失 ， 儿子 ， ，


Input:  有 人 感激 过 你 的 善良 吗 ， 貌似 他们 只 会 得寸进尺 。 
Output: 有 感激 感激 感激 感激 吗 善良 吗 吗 吗


Input:  全球 移动 互联网 大会 开始 啦 ！ 
Output: 转让 本人 2 小米 2 手机 2 2 意者 私聊 私聊 263996097 263996097 263996097 263996097 263996097 263996097 263996097 263996097 263996097


Input:  旁边 那个 二 货 我 不 认识 它 。 （ 图 转 ） 
Output: 旁边 那个 货 货 的 货


Input:  BB 满 月 是 30 天 还 是 31 天 ？ 
Output: 满月 按照 按照 农历 算 算 算


Input:  你们 的 妈妈 也 会 把 你们 的 卧室 叫做 ‘ 狗窝 ’ 吗 ？ 
Output: 你 妈妈 是 的 妈妈 的


Input:  八字桥 ， 这 才 是 真正 的 江南 style 。 晚安 ， 绍兴 。 
Output: 据说 ， ， ， 早 早 ， 立交桥


Input:  我们 的 传奇 星 越来越 茁壮 了 ⋯ 谢谢 大家 的 支持 ！ alink 
Output: 好 那里 的 ， 的 惬意


Input:  怎么 看 也 不 像 失恋 33 天啊 ， 看 这个 笑容 像 恋爱 33 天 
Output: 这 类 菊花 造型 造型 加上 加上


Input:  刚才 摘录 并 支持 胡 在 十八 大 讲话 内容 的 贴 果然 被 平碧 了 么 … … 
Output: 丫 电脑 电脑 中毒 ， ， 自动 自动


Input:  要不是 看在 你 妈 养 我 的 份上 ， 我 早挠 你 了 ！ 
Output: 这 视频 ， 没完 ， 吃


Input:  上 周 第一 批 笔试 结束 被 铺天盖地 的 试卷 包围 了

Output: 其实 点 联系 联系 只喺唔 知 知 开口


Input:  发表 了 博文  《 芭莎 电影 创刊号 —— 李冰冰 》  - alink 
Output: 我 每 ， ， ， ， 了


Input:  一 场 秋雨 一 重色 ， 红叶 黄花 秋景宽 。 早上 好 ， 各位 ！ 
Output: 我 让 ， 一辈子 一辈子 呆在 呆在 呆在 呆在 上帝 上帝 上帝 上帝 上帝 上帝 上帝 上帝 上帝 上帝 上帝


Input:  突然 发现 ： 现在 的 生活 才 叫 “ 乐活 ” ！ 
Output: 只要 愿意 的 的 的 的


Input:  # 颜色# 有种 夏天 叫做 孔雀蓝 ！ 
Output: 还是 最爱 以前 颜色 的 形式


Input:  # 郑在晒 晚餐 # 吃 晚饭 了 吗 ？ 在家 吃 还是 在 外边 吃 ？ 
Output: 想问 郑 大哥 大哥 的 的


Input:  我 越来越 失去 了 礼让 行人 的 美德 我 在 这里 : alink 
Output: 欸 ， 可是 可是 可是 马路 行 老是 让人


Input:  英国 BBC 评出 的 人生 50 个 必 去 圣地 。 有生之年 ， 谁 能 全部 走遍 ？ 
Output: 有生之年 ， 塘 走遍 ， 个遍


Input:  等到 我 老 了 ， 我 就 想 住 这样 的 房子 ！ 。 
Output: 老 ， ， ， 破 建 了


Input:  松花江 畔 ， 灯光 闪闪 ， 波光粼粼 ， 清风 徐徐 。 
Output: 欢迎贾 老师 ， ， ， 吉林 ，


Input:  坚决 反对 对 民主 的 朝鲜 指手画脚 ， 干涉 禅让 。 
Output: 这 可不是 禅让 ， ， 接班




In [26]:
eval_post_resp = []
for i in range(30):
    eval_post_resp.append([' '.join(input_seqs[i]), ' '.join(output_seqs[i])])
beam_size = 1
beam_search_test(encoder, decoder, decoder_init_state, eval_post_resp, ctx, max_seq_len)

Input:  中国 移动 营销 行来 发展 报告 alink <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Output: 
<bos> 真 ， 多 ， ， ， <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 537.044742584
Expect: 王 大姐 ， 打字 细心 一点 <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

Input:  小马 也 疯狂 ------ 地位 之 争 。 <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Output: 
<bos> 这 不 不 的 句 会 <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 531.798723221
Expect: 于 老师 不 给 劝劝 架么 告诉 他们 再 挣 也 不 是 老大 <eos> <pad> <pad> <pad> <pad> <pad> 

Input:  那些 年 ， 我们 一起 偷看 过 的 电视 。 「 暴 走 漫画 」 <eos> <pad> <pad> <pad> <pad>
Output: 
<bos> 这 走 是 ， 的 的 <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 543.554569244
Expect: 真不愧是 这么 走 出来 的 爹······· <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

Input:  北京 的 小 纯洁们 ， 周日见 。 # 

Output: 
<bos> 这 上辈子 ， ， ， ， <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 549.200883865
Expect: 我 喜欢 ； 水能 复舟 。 也 能 。 <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

Input:  # 米吧秀图 # 月 底 了 ， 敢 说出 你 手机 还 有 多少 流量 么 ？ <eos> <pad> <pad>
Output: 
<bos> 这个 ， 邓丽君 ， 的 … <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 550.253286362
Expect: 这个 图片 是 谁 照 的 啊 ， 真是 绝啊 <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 



In [17]:
def beam_search_test(encoder, decoder, decoder_init_state, eval_post_resp, ctx, max_seq_len):
    for p_r in eval_post_resp:
        print('Input: ', p_r[0])
        input_tokens = p_r[0].split(' ') + [EOS]
        while len(input_tokens) < max_seq_len:
            input_tokens.append(PAD)
        inputs = nd.array(input_vocab.to_indices(input_tokens), ctx = ctx)
        encoder_state = encoder.begin_state(func=mx.nd.zeros, batch_size=1, ctx=ctx)
        encoder_outputs, encoder_state = encoder(inputs.expand_dims(0), encoder_state)
        encoder_outputs = encoder_outputs.flatten()
        
        #decoder_input = nd.array([output_vocab.token_to_idx[BOS]], ctx=ctx)
        decoder_state = decoder_init_state(encoder_state[0])
        
        candidates = [[BOS] for _ in range(beam_size)]
        probs = [0.0 for _ in range(beam_size)]
        
        for _ in range(max_output_len):
            tmp = []
            tmp_cand = []
            for k in range(beam_size):
                decoder_input = nd.array([output_vocab.token_to_idx[candidates[k][-1]]], ctx=ctx)
                decoder_output, decoder_state = decoder(decoder_input, decoder_state, encoder_outputs)
                
                pred_score, pred_index = decoder_output.topk(ret_typ='both',k=beam_size)
                #pred_i = int(pred_index[0].asnumpy()[0])
                for j in range(beam_size):
                    tmp.append(probs[k] + pred_score[0].asnumpy()[j])
                    tmp_cand.append(candidates[k] + [output_vocab.idx_to_token[int(pred_index[0].asnumpy()[j])]])
            top_k_idx = np.argsort(tmp)[-beam_size:]
            for k in range(beam_size):
                candidates[k] = tmp_cand[top_k_idx[k]]
                probs[k] = tmp[top_k_idx[k]]
                #print(' '.join(candidates[k]))
        #print(probs)
        top_idx = np.argsort(probs).tolist()
        #print(top_idx)
        print('Output: ')
        for idx in top_idx:
            print(' '.join(candidates[idx]), probs[idx])
        print('Expect:', p_r[1], '\n')

In [31]:
encoder.collect_params().save('encoder-bless-L118.params')
decoder.collect_params().save('decoder-bless-L118.params')
decoder_init_state.collect_params().save('decoder-init-state-bless-L118.params')